In [1]:
## Dataset construction

In [2]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])

base


## Evaluation

In [3]:
import pandas as pd
data = pd.read_csv("./Stage3.csv")
data = data[data['C_NMR'] != 'No NMR data was extracted from the provided text.']

In [4]:
data.head()

,Unnamed: 0,Molecule Index,SMILES,Formula,CoT,fragments pool,H_NMR,C_NMR
0,0,5_99\n,CCCCC1=CC=CC=C1,C10H14,['To analyze the potential structures for the ...,1. C1=CC=C(C=C1)C\n2. C=C1C=CC=CC1\n3. C1=CC=C...,"0.87 (3H, t, J = 6.5 Hz), 1.30 (2H, tq, J = 6....","14.0 (1C, s), 22.6 (1C, s), 32.8 (1C, s), 35.5..."
1,1,5_98\n,CCCCOCCCC,C8H18O,['To analyze the IR and mass spectra for a mol...,C8H18O could produce the following fragments b...,"0.88 (6H, t, J = 7.1 Hz), 1.40 (4H, h, J = 7.1...","14.02 (2C, s), 19.15 (2C, s), 31.1 (2C, s), 71..."
2,2,5_97\n,CCC[N+](=O)[O-],C3H7NO2,"[""To analyze the provided IR and Mass spectra ...",1. CC(N)C(=O)O\n2. CNCC(=O)O\n3. NCC(=O)O\n4. ...,"1H NMR: 2.24 (3H), 5.95 (1H, dd, J = 10.9, 1.5...","13.72 (1C, s), 29.2 (1C, s), 71.7 (1C, s)"
3,3,5_96\n,C1=CC=C(C=C1)CCCBr,C9H11Br,"[""To analyze the molecular structure of a comp...",1. c1ccccc1\n2. c1ccccc1Br\n3. CCC\n4. CC(C),"2.11 (2H, tt, J = 7.3, 6.8 Hz), 2.62 (2H, t, J...","5.95 (1H, dd, J = 10.9, 1.5 Hz), 6.29-6.54 (2H..."
4,4,5_95\n,C[C@H]([C@@H](C(=O)O)N)O,C4H9NO3,['To analyze and determine potential structure...,1. `CC(C(=O)O)N`\n2. `C(C(C(=O)O)N)O`\n3. `CC(...,"1.19 (3H, d, J = 6.2 Hz), 3.66-3.80 (2H, 3.73 ...","18.7 (1C, s), 61.6 (1C, s), 67.1 (1C, s), 174...."


In [ ]:
import pandas as pd
import os
iteration = 3

ratios = {
        'Molecule generation':1
    }
for i in range(0, iteration):
    total_samples = 100
    samples_per_class = {clss: int(total_samples * ratio) for clss, ratio in ratios.items()}
    print(samples_per_class)

    sampled_data = pd.DataFrame()
    for clss, n_samples in samples_per_class.items():
        print(clss)
        sampled_class_data = data.sample(n=100)
        sampled_data = pd.concat([sampled_data, sampled_class_data])


    if len(sampled_data) < total_samples:
        additional_samples = data[~data.index.isin(sampled_data.index)].sample(n=total_samples - len(sampled_data), random_state=42)
        sampled_data = pd.concat([sampled_data, additional_samples])
    os.makedirs('./data/mol_figures/mol_understanding', exist_ok=True)
    sampled_data.to_csv(f'./data/mol_figures/step3/C_NMR_sampled_questions_answers_{i}.csv', index=False)
    print("Sampled data saved to 'sampled_questions_answers.csv'")

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_path, tokenizer_path=None, device='cuda:0', **kwargs):
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        **kwargs
    ).to(device).eval()

    tokenizer_path = model_path if tokenizer_path is None else tokenizer_path

    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path,
        trust_remote_code=True,
        use_fast=False,
        padding_side='left',
    )
    tokenizer.pad_token = tokenizer.unk_token
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'
    return model, tokenizer

def llm_generate(inputs, conv_template, model, tokenizer, batch_size=80, random_sample=False, max_new_tokens=256):
    input_ids_list = []
    for i in range(len(inputs)):
        conv_template.append_message(conv_template.roles[0], inputs[i])
        conv_template.append_message(conv_template.roles[1], None)
        prompt = conv_template.get_prompt()
        encoding = tokenizer(prompt)
        toks = encoding.input_ids
        input_ids = torch.tensor(toks).to(model.device)
        input_ids_list.append(input_ids)
        conv_template.messages = []
    pad_tok = tokenizer.pad_token_id
    max_input_length = max([ids.size(0) for ids in input_ids_list])
    # Pad each input_ids tensor to the maximum length
    padded_input_ids_list = []
    for ids in input_ids_list:
        pad_length = max_input_length - ids.size(0)
        padded_ids = torch.cat([torch.full((pad_length,), pad_tok, device=model.device), ids], dim=0)
        padded_input_ids_list.append(padded_ids)
    input_ids_tensor = torch.stack(padded_input_ids_list, dim=0)
    attn_mask = (input_ids_tensor != pad_tok).type(input_ids_tensor.dtype)
    generation_config = model.generation_config
    generation_config.max_new_tokens = max_new_tokens
    if random_sample:
        generation_config.do_sample = True
        generation_config.temperature = 0.7
        generation_config.top_p = 0.9
    else:
        generation_config.do_sample = False
        generation_config.temperature = None
        generation_config.top_p = None
    flag = False
    while not flag:
        try:
            output_ids_new = []
            for i in range(0, len(input_ids_tensor), batch_size):
                input_ids_tensor_batch = input_ids_tensor[i:i + batch_size]
                attn_mask_batch = attn_mask[i:i + batch_size]
                output_ids_batch = model.generate(input_ids_tensor_batch,
                                                  attention_mask=attn_mask_batch,
                                                  generation_config=generation_config,
                                                  pad_token_id=tokenizer.pad_token_id)

                for j in range(len(output_ids_batch)):
                    output_ids_new.append(output_ids_batch[j][max_input_length:])
            flag = True
        # except cuda out of memory error
        except torch.cuda.OutOfMemoryError:
            batch_size = batch_size // 2
    gen_strs = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in output_ids_new]
    return gen_strs

## few-shot evaluation

In [7]:
import openai
from openai import OpenAI
import anthropic
import os
from fastchat.model.model_adapter import get_conversation_template
from utils import *
import pandas as pd
# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = ''
os.environ['ANTHROPIC_API_KEY'] = ''

# Define the prompt
prompt = """
As an expert organic chemist, your task is to analyze and determine the one potential structures that can be selected as the starting points for the molecular construction\
Utilize your knowledge to systematically explore and identify plausible structural configurations based on the fragment pools provided.
Do the analysis internally and only output the final SMILES.
"""
cache_dir = "/scratch365/kguo2/TRANS_cache/"

def get_llm_response(model_name,model,prompt):
    if 'gpt' in model_name:
        openai.api_key = os.getenv("OPENAI_API_KEY")
        client = OpenAI()
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are an expert organic chemist."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        ).choices[0].message.content
    elif 'claude' in model_name:
        client = anthropic.Anthropic()
        response = client.messages.create(
            max_tokens=1024,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="claude-3-opus-20240229",
        )
    else:
        conv_template = get_conversation_template(model_paths[model_name])
        conv_template.system_prompt = "You are an expert organic chemist."
        response = llm_generate(prompt, conv_template, model, tokenizer, batch_size=10, random_sample=True, max_new_tokens=1024)
    return response

def generate_prompt(prompt):
    prompt = prompt + 'Target C-NMR Spectrum: ' + row['C_NMR'] + '\n' + 'fragments pool:'+ '\n' + row['fragments pool'] + '\n' + "output:"
    return prompt

def is_open_source(model_name):
    if 'claude' in model_name or 'gemini' in model_name or 'gpt' in model_name:
        return False
    return True


In [8]:
os.environ['TRANSFORMERS_CACHE'] = "/scratch365/kguo2/TRANS_cache/"
os.environ['HF_HUB_CACHE'] = "/scratch365/kguo2/TRANS_cache/"
os.environ['HF_HOME'] = "/scratch365/kguo2/TRANS_cache/"
os.environ['HUGGINGFACE_HUB_CACHE']="/scratch365/kguo2/TRANS_cache/"
os.environ['HF_DATASETS_CACHE']="/scratch365/kguo2/TRANS_cache/"
print(os.environ['HF_HUB_CACHE'])

/scratch365/kguo2/TRANS_cache/


In [9]:
import pandas as pd
data = pd.read_csv("./data/mol_figures/step3/step3.csv")
questions = []
for _, row in data.iterrows():
    question = generate_prompt(prompt)
    questions.append(question)


In [10]:
total_length = sum(len(item) for item in questions)

# Calculate the mean length
mean_length = total_length / len(questions) if questions else 0 
mean_length

590.0655021834061

In [ ]:
! huggingface-cli login

跑下面哪个框框！！！！

In [ ]:
iteration = 3
model_names = ['gpt-3.5-turbo', 'gpt-4o', 'claude']

for model_name in model_names:
    for i in range(0, iteration):
        question_data = pd.read_csv(f'./data/mol_figures/step3/C_NMR_sampled_questions_answers_{i}.csv')
        if is_open_source(model_name) and i == 0:
            print(model_name)
            model, tokenizer = load_model_and_tokenizer(model_paths[model_name],
                                                        cache_dir=cache_dir,
                                                        low_cpu_mem_usage=True,
                                                        use_cache=False,
                                                        device='cuda')
            # print('running')
                                                    
        data_frame = pd.DataFrame(columns=["Molecule Index", "Generated Response"])
        prompts = []
        for index, row in question_data.iterrows():
            prompt = """
            As an expert organic chemist, your task is to analyze and determine the one potential structures that can be selected as the starting points for the molecular construction.Utilize your knowledge to systematically explore and identify plausible structural configurations based on the fragment pool provided.Do the analysis internally and only output the final SMILES.
            Here is an example:
            Target C-NMR Spectrum: 31.1 (6C, s), 34.7 (2C, s), 112.3 (2C, s), 119.3 (1C, s), 152.6 (2C, s), 155.3 (1C, s)
            Fragments pool: C=C
                            C1=CC=CC=C1
                            CC
                            CCC
                            CC(C)C
                            CC(C)(C)C
                            O
                            C=C-C
                            C=C-CC
                            C=C-C(O)
                            C=C-C(C)
            output: C=C-C,
                    C=C-C(C)
                    """
            prompt = generate_prompt(prompt)
            prompts.append(prompt)
        if is_open_source(model_name):
            generated_responses = get_llm_response(model_name,model, prompts)
            for index, row in question_data.iterrows():
                data_frame.loc[len(data_frame)] = [row['Molecule Index'], generated_responses[index]]
        else:
            for index, row in question_data.iterrows():
                generated_response = get_llm_response(model_name,_, prompt)
                if 'claude' in model_name:
                    generated_response = generated_response.content[0].text
                data_frame.loc[len(data_frame)] = [row['Molecule Index'], generated_response]
                data_frame.to_csv(f'./data/mol_figures/step3/C_NMR/{model_name}_generated_responses_{i}.csv', index=False)
                print(index, generated_response)
    if is_open_source(model_name):
        del model, tokenizer

In [ ]:
iteration = 3
model_names = ['llama3','vicuna','mistral']

for model_name in model_names:
    for i in range(0, iteration):
        question_data = pd.read_csv(f'./data/mol_figures/step3/C_NMR_sampled_questions_answers_{i}.csv')
        # question_data = question_data.sample(1)
        if is_open_source(model_name) and i == 0:
            print(model_name)
            model, tokenizer = load_model_and_tokenizer(model_paths[model_name],
                                                        cache_dir=cache_dir,
                                                        low_cpu_mem_usage=True,
                                                        use_cache=False,
                                                        device='cuda')
            # print('running')
                                                    
        data_frame = pd.DataFrame(columns=["Molecule Index", "Generated Response"])
        prompts = []
        for index, row in question_data.iterrows():
            prompt = """
            As an expert organic chemist, your task is to analyze and determine the one potential structures that can be selected as the starting points for the molecular construction.Utilize your knowledge to systematically explore and identify plausible structural configurations based on the fragment pool provided.Do the analysis internally and only output the final SMILES.
            Here is an example:
            Target C-NMR Spectrum: 31.1 (6C, s), 34.7 (2C, s), 112.3 (2C, s), 119.3 (1C, s), 152.6 (2C, s), 155.3 (1C, s)
            Fragments pool: C=C
                            C1=CC=CC=C1
                            CC
                            CCC
                            CC(C)C
                            CC(C)(C)C
                            O
                            C=C-C
                            C=C-CC
                            C=C-C(O)
                            C=C-C(C)
            output: C=C-C,
                    C=C-C(C)
                    """
            prompt = generate_prompt(prompt)
            prompts.append(prompt)
        if is_open_source(model_name):
            generated_responses = get_llm_response(model_name,model, prompts)
            for index, row in question_data.iterrows():
                data_frame.loc[len(data_frame)] = [row['Molecule Index'], generated_responses[index]]
        else:
            for index, row in question_data.iterrows():
                generated_response = get_llm_response(model_name,_, prompt)
                if 'claude' in model_name:
                    generated_response = generated_response.content[0].text
                # print(generated_response)
                data_frame.loc[len(data_frame)] = [row['Molecule Index'], generated_response]
        data_frame.to_csv(f'./data/mol_figures/step3/C_NMR/{model_name}_generated_responses_{i}.csv', index=False)
    if is_open_source(model_name):
        del model, tokenizer
        

In [ ]:
import numpy as np
model_names = ['llama3','vicuna', 'mistral']
for model_name in model_names:
    print(model_name)
    sa_f1 = []
    sa_acc = []
    ar_acc = []
    ar_f1 = []
    fg_acc = []
    fg_f1 = []
    matchs = []
    for i in range(0, 3):
        generated_answers = pd.read_csv(f'./data/mol_figures/mol_understanding/cot_{model_name}_generated_responses_{i}.csv')
        results = evaluate_responses(generated_answers)
        sa_degree_results = evaluate_saturation_degree(generated_answers)
        sa_f1.append(results['Saturation']['F1 Score'])
        sa_acc.append(results['Saturation']['Accuracy'])
        ar_acc.append(results['Aromatic Rings']['Accuracy'])
        ar_f1.append(results['Aromatic Rings']['F1 Score'])
        fg_acc.append(results['Functional Group']['Accuracy'])
        fg_f1.append(results['Functional Group']['F1 Score'])
        matchs.append(sa_degree_results['Saturation degree']['Accuracy'])
    sa_f1 = np.array(sa_f1)
    sa_acc = np.array(sa_acc)
    ar_acc = np.array(ar_acc)
    ar_f1 = np.array(ar_f1)
    fg_acc = np.array(fg_acc)
    fg_f1 = np.array(fg_f1)
    matchs = np.array(matchs)
    print(sa_f1.mean(),ar_f1.mean(),fg_f1.mean())
    print(sa_f1.std(),ar_f1.std(),fg_f1.std())
    print(sa_acc.mean(),ar_acc.mean(),fg_acc.mean(),matchs.mean())
    print(sa_acc.std(),ar_acc.std(),fg_acc.std(),matchs.std())
    print("f1 mean",(sa_f1.mean() * 0.2 + ar_f1.mean() * 0.3 + fg_f1.mean() * 0.3 ) / 0.8)
    print("f1 std",(sa_f1.std() * 0.2 + ar_f1.std() * 0.3 + fg_f1.std() * 0.3 ) / 0.8)
    print("acc mean",(sa_acc.mean() * 0.2 + ar_acc.mean() * 0.3 + fg_acc.mean() * 0.3 ) / 0.8)
    print("acc std",(sa_acc.std() * 0.2 + ar_acc.std() * 0.3 + fg_acc.std() * 0.3 ) / 0.8)